In [2]:
import pandas as pd
from time import sleep
from pytz import timezone
from datetime import datetime, timedelta
from user_profiling_layer.preferences_management_module import *
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
interactions = pd.DataFrame(columns=[
    'telegram_user',
    'group',
    'total_num', 
    'num_past_week', 
    'num_days_past_week', 
    'days_past_week',
    'all'
])

In [ ]:
users = get_all_users_from_db()

for telegram_user, mfp_user, _, sender_id, user_group, user_timezone in users:

    user_query = f'''
        SELECT JSON_UNQUOTE(JSON_EXTRACT(data, '$.timestamp')) AS timestamp
        FROM events
        WHERE JSON_EXTRACT(data, '$.sender_id') = '{sender_id}'
        AND JSON_EXTRACT(data, '$.event') = 'user'
        AND JSON_EXTRACT(data, '$.metadata.message.text') NOT LIKE '/%';
    '''

    with mysql.connector.connect(host='localhost', db='philhumans', user='root', password='root') as connection:
        with connection.cursor() as cursor:
            cursor.execute(user_query)
            rows = cursor.fetchall()

    timestamps = [datetime.fromtimestamp(float(row[0])) for row in rows if row]
    timestamps.sort(reverse=True)
    
    if timestamps:
        total_num = len(timestamps)
        
        now = datetime.now()
        one_week_ago = now - timedelta(days=8)
        
        # Filter timestamps from the last week
        timestamps_past_week = [ts for ts in timestamps if ts >= one_week_ago]
        num_past_week = len(timestamps_past_week)
        
        # Find unique days of interaction in the last week
        days_past_week = set(ts.date().strftime('%m-%d') for ts in timestamps_past_week)
        num_days_past_week = len(days_past_week)
        
        # Convert set to list for recording
        days_past_week = sorted(list(days_past_week))
        
        # Create a DataFrame row with the calculated metrics
        new_row = pd.DataFrame([{
            'telegram_user': telegram_user,
            'group': user_group,
            'total_num': total_num,
            'num_past_week': num_past_week,
            'num_days_past_week': num_days_past_week,
            'days_past_week': days_past_week,
            'all': timestamps
        }])
        
        # Concatenate the new row to the original DataFrame
        interactions = pd.concat([interactions, new_row], ignore_index=True)
    else:
        # If there are no timestamps, add a row with zeros and empty list
        new_row = pd.DataFrame([{
            'telegram_user': telegram_user,
            'group': user_group,
            'total_num': 0,
            'num_past_week': 0,
            'num_days_past_week': 0,
            'days_past_week': [],
            'all': []
        }])
        
        # Concatenate the new row to the original DataFrame
        interactions = pd.concat([interactions, new_row], ignore_index=True)

In [5]:
interactions.to_csv('interactions.csv', index=False)

In [ ]:
interactions[interactions['num_days_past_week']<5].sort_values('num_days_past_week')

In [7]:
def print_users_by_interaction_days(df):
    for days in range(5):
        users_with_days_of_interaction = df[df['num_days_past_week'] == days]['telegram_user'].tolist()
        print(f"Number of users with {days} days of interaction in the past week: {len(users_with_days_of_interaction)}")
        print(f"Users: {users_with_days_of_interaction}\n")

In [ ]:
print_users_by_interaction_days(interactions)

In [ ]:
colors = {
    '1': 'lightcoral',
    '2': 'sandybrown',
    '3': 'khaki'
}

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=interactions, x='group', y='total_num', palette=colors)
plt.title('Distribution of Total Interactions per Group')
plt.xlabel('Group')
plt.ylabel('Total Number of Interactions')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=interactions, x='group', y='num_days_past_week', palette=colors)
plt.title('Distribution of Number of Days of Interaction in the Past Week per Group')
plt.xlabel('Group')
plt.ylabel('Number of Days of Interaction in the Past Week')
plt.show()

In [ ]:
group_interaction_sums = interactions.groupby('group')['total_num'].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=group_interaction_sums, x='group', y='total_num', palette=colors)
plt.title('Total Number of Interactions per Group')
plt.xlabel('Group')
plt.ylabel('Total Number of Interactions')
plt.show()

In [ ]:
group_interaction_avg = interactions.groupby('group')['total_num'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=group_interaction_avg, x='group', y='total_num', palette=colors)
plt.title('Average Number Interactions per Group')
plt.xlabel('Group')
plt.ylabel('Average Number of Interactions')
plt.show()

In [ ]:
group_days_sums = interactions.groupby('group')['num_days_past_week'].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=group_days_sums, x='group', y='num_days_past_week', palette=colors)
plt.title('Total Number of Days of Interaction in the Past Week per Group')
plt.xlabel('Group')
plt.ylabel('Total Number of Days of Interaction in the Past Week')
plt.show()